Mini Project: Intelligent Travel Planning Chatbot
In this project, you will build an intelligent, multi-turn chatbot that helps users plan a customized trip through conversation. The chatbot should be able to guide the user step-by-step, collect important travel information, and finally generate a structured travel plan in JSON format.

The goal is not just to create a chatbot that replies — but one that follows a conversational flow, knows when to stop asking questions, and return a structured output that can be reused or integrated into other systems.

In [10]:
import os
import getpass
import json
from pprint import pprint
from typing import Sequence, List
from typing_extensions import Annotated, TypedDict

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)

In [12]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key:  ········


In [13]:
class TravelPlan(BaseModel):
  destination: str
  date: str
  budget: str
  preferences: List[str]
  itinerary_suggestions: List[str]  # AI-generated itinerary suggestions
  travel_tips: List[str]            # AI-generated travel tips

In [14]:
python_output_parser = PydanticOutputParser(pydantic_object=TravelPlan)

In [17]:
trimmer = trim_messages(
    max_tokens=7000,
    strategy="last",
    token_counter= init_chat_model("gpt-4o", model_provider="openai", streaming=True),
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [23]:
format_instructions = python_output_parser.get_format_instructions().replace("{", "{{").replace("}", "}}")


system_prompt = f"""
You are a helpful travel planning assistant.

## Objective:
Guide the user through a short conversation to collect the following four details, one at a time, in order to create a personalized travel plan:
1. Destination
2. Travel Date
3. Budget
4. Preferences (e.g., food, museums, nature, shopping, culture)

## Interaction Guidelines:
- Be friendly and concise.
- Ask one question at a time.
- Wait for the user's response before moving to the next question.
- If an answer is unclear or incomplete, ask for clarification or rephrase.

## Memory Rules:
- Keep track of which details are already provided.
- Do not ask for the same information twice.

## Completion Steps:
When all four details are collected:
1. Summarize the gathered information.
2. Ask if the user wants to revise anything.
3. If confirmed, generate a travel plan strictly in the format below:

Respond ONLY with a valid JSON object using this structure:

{format_instructions}

## Notes:
- Do not include any explanation or text outside the JSON object.
- If the user hasn’t finished, continue the conversation accordingly.
"""

In [24]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
])

In [ ]:
# LangGraph state schema
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Node logic that streams output and checks for final response
def call_model(state: State):
    trimmed = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed})
    chunks = init_chat_model("gpt-4o", model_provider="openai", streaming=True).stream(prompt)
    final = ""
    print("\nAI:", end=" ", flush=True)
    for chunk in chunks:
        if isinstance(chunk, AIMessage):
            print(chunk.content, end="", flush=True)
            final += chunk.content
    print()
    return {"messages": [AIMessage(content=final)]}

# Build LangGraph
workflow = StateGraph(state_schema=State)
workflow.add_node("model", call_model)
workflow.set_entry_point("model")

# Memory management
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Conversation initialization
conversation = [SystemMessage(content=system_prompt)]
config = {"configurable": {"thread_id": "travel123"}}

# All above steps are set up, now we can start the interactive conversation.
print("Travel Planning Chatbot\n")

# Assistant greeting
result = app.invoke({"messages": conversation}, config=config)
assistant_response = result["messages"][-1]
conversation.append(assistant_response)

# Interactive loop
while True:
    user_input = input("\nYou: ")
    conversation.append(HumanMessage(content=user_input))
    result = app.invoke({"messages": conversation}, config=config)
    assistant_response = result["messages"][-1]
    conversation.append(assistant_response)

    try:
        # Try to parse the final response
        travel_plan = output_parser.invoke(assistant_response.content)

        print("\nFinal Structured Travel Plan:")
        pprint(travel_plan.model_dump(), sort_dicts=False, compact=True)

        # Save as JSON
        with open("travel_plan.json", "w", encoding="utf-8") as f:
            json.dump(travel_plan.model_dump(), f, indent=2, ensure_ascii=False)
        print("\nTravel plan saved to 'travel_plan.json'")
        break

    except Exception:
        # Not a structured response yet, continue conversation
        continue

Travel Planning Chatbot


AI: Hi there! To help you create a personalized travel plan, can you first tell me your desired destination?



You:  Mysore



AI: Great choice! When are you planning to travel to Mysore?



You:  Next week



AI: Sounds exciting! Could you let me know your budget for the trip?


...


AI: Thank you! Lastly, could you share your preferences? For example, are you interested in food, museums, nature, shopping, or culture?



You:  experiences. and fancy hotels



AI: Great! Here’s a summary of your travel plan:

- **Destination:** Mysore
- **Travel Date:** Next week
- **Budget:** 20k
- **Preferences:** Experiences, Fancy hotels

Would you like to revise any of these details before I create your travel plan?



You:  no



AI: ```json
{
    "destination": "Mysore",
    "date": "Next week",
    "budget": "20k",
    "preferences": ["Experiences", "Fancy hotels"],
    "itinerary_suggestions": [
        "Visit the Mysore Palace and witness its grandeur.",
        "Experience a luxurious stay at a heritage hotel like the Lalitha Mahal Palace.",
        "Indulge in a traditional Mysorean meal at a renowned local restaurant.",
        "Explore the beautiful Brindavan Gardens during the evening light show.",
        "Visit Chamundi Hill for a panoramic view of the city."
    ],
    "travel_tips": [
        "Book your hotels in advance to secure the best deals.",
        "Consider hiring a local guide for a more in-depth experience.",
        "Try local sweets like Mysore Pak at popular sweet shops.",
        "Use apps like Ola or Uber for convenient travel within the city.",
        "Check